In [81]:
#Let's train some models on this data

In [104]:
import pandas as pd
import numpy as np
import os
import sys 
sys.path.append(os.path.dirname(os.getcwd()))
import warnings
import config as cfg
from sklearn.linear_model import LogisticRegression #importing for Logistic regression
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from sklearn.model_selection import train_test_split
import random
import lightgbm as lgb
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import joblib

warnings.filterwarnings("ignore")


In [83]:
#importing libraries
data_df=pd.read_csv(cfg.input+"pre_processed.csv",index_col=0)
data_df.head(4)

,FCVC,NCP,CH2O,FAF,TUE,BMI,Target,Gender_Male,family_history_with_overweight_yes,FAVC_yes,...,CAEC_no,SMOKE_yes,SCC_yes,CALC_Sometimes,CALC_no,MTRANS_Bike,MTRANS_Motorbike,MTRANS_Public_Transportation,MTRANS_Walking,log_age
0,2.000000,2.983297,2.763573,0.000000,0.976473,28.259565,6,True,True,True,...,False,False,False,True,False,False,False,True,False,3.196344
1,2.000000,3.000000,2.000000,1.000000,1.000000,23.422091,1,False,True,True,...,False,False,False,False,True,False,False,False,False,2.890372
2,1.880534,1.411685,1.910378,0.866045,1.673584,17.126706,0,False,True,True,...,False,False,False,False,True,False,False,True,False,2.890372
3,3.000000,3.000000,1.674061,1.467863,0.780199,44.855798,4,False,True,True,...,False,False,False,True,False,False,False,True,False,3.042269


In [84]:
#splitting the data
scale=StandardScaler()

X,Y=data_df.drop('Target',axis=1),data_df['Target']
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2)
X_train=scale.fit_transform(X_train)
X_test=scale.transform(X_test)

In [85]:
#Creating a random model for baseline value
def random_pred(df):
    return random.choice([0,1,2,3,4,5,6])
Y_rand_pred=pd.DataFrame(X_train).apply(random_pred,axis=1)

In [86]:
#Let's see the accuracy of random model
acc_rand=accuracy_score(Y_train,Y_rand_pred)
print("Accuracy of Random predictory:",acc_rand)

Accuracy of Random predictory: 0.14265927977839335


* So now we know that our models needs to outperform atleast this score

In [87]:
log_reg=LogisticRegression(multi_class='ovr')
log_reg.fit(X_train,Y_train)
Y_train_log=log_reg.predict(X_train)
Y_test_log=log_reg.predict(X_test)
acc_log_train=accuracy_score(Y_train,Y_train_log)
acc_log_test=accuracy_score(Y_test,Y_test_log)
print("Accuracy of logistic regression on train set:",acc_log_train)
print("Accuracy of logistic regression on testing set:",acc_log_test)

Accuracy of logistic regression on train set: 0.7621341683728773
Accuracy of logistic regression on testing set: 0.7683044315992292


* Well! Not Bad!!

In [88]:
#Training it on SVC
svc=OneVsRestClassifier(SVC()).fit(X_train,Y_train)
Y_train_svc=svc.predict(X_train)
Y_test_svc=svc.predict(X_test)
acc_svc_train=accuracy_score(Y_train,Y_train_svc)
acc_svc_test=accuracy_score(Y_test,Y_test_svc)
print("Accuracy of SVC on train set:",acc_svc_train)
print("Accuracy of SVC on test set:",acc_svc_test)

Accuracy of SVC on train set: 0.8335541370588944
Accuracy of SVC on test set: 0.8005780346820809


* Yo! This seems to be even better

In [89]:
rf=RandomForestClassifier(n_estimators=10,random_state=0,oob_score=True)
rf.fit(X_train,Y_train)
Y_train_rf=rf.predict(X_train)
Y_test_rf=rf.predict(X_test)
acc_rf_train=accuracy_score(Y_train,Y_train_rf)
acc_rf_test=accuracy_score(Y_test,Y_test_rf)
print("Accuracy of RF on train set:",acc_rf_train)
print("Accuracy of RF on test set:",acc_rf_test)

Accuracy of RF on train set: 0.993436107431049
Accuracy of RF on test set: 0.8672928709055877


In [90]:
Y.value_counts()

Target
4    4046
3    3248
1    3082
2    2910
0    2523
6    2522
5    2427
Name: count, dtype: int64

In [95]:
len(X_train[0])

21

#### Clearly this is overfitting

#### How about using Boosting techniques: Let's see for XGBoost

In [91]:
xgb_clf=xgb.XGBClassifier(objective="multi:softmax",max_depth=3,n_estimators=60).fit(X_train,Y_train)
Y_train_xgb=xgb_clf.predict(X_train)
Y_test_xgb=xgb_clf.predict(X_test)
acc_xgb_train=accuracy_score(Y_train,Y_train_xgb)
acc_xgb_test=accuracy_score(Y_test,Y_test_xgb)
print("Accuracy of XGB on train set:",acc_xgb_train)
print("Accuracy of XGB on test set:",acc_xgb_test)

Accuracy of XGB on train set: 0.8986510899674817
Accuracy of XGB on test set: 0.884393063583815


#### Woah! it improved

In [98]:
### Time to call mighty ANN
## Defining ann

ann=Sequential()
ann.add(Dense(units=21,activation='relu'))
ann.add(Dense(units=21,activation='relu'))
ann.add(Dense(units=7,activation='softmax'))

#Compile the trained model
ann.compile(optimizer='rmsprop',
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy'])
ann.fit(x=X_train,y=Y_train,epochs=50,batch_size=50,validation_data=(X_test,Y_test))


Epoch 1/50
333/333 [==============================] - 4s 8ms/step - loss: 1.2547 - accuracy: 0.5208 - val_loss: 0.8334 - val_accuracy: 0.6676
Epoch 2/50
333/333 [==============================] - 2s 5ms/step - loss: 0.7156 - accuracy: 0.7247 - val_loss: 0.6042 - val_accuracy: 0.7688
Epoch 3/50
333/333 [==============================] - 2s 5ms/step - loss: 0.5603 - accuracy: 0.7909 - val_loss: 0.4945 - val_accuracy: 0.8237
Epoch 4/50
333/333 [==============================] - 2s 5ms/step - loss: 0.4950 - accuracy: 0.8212 - val_loss: 0.4531 - val_accuracy: 0.8382
Epoch 5/50
333/333 [==============================] - 2s 5ms/step - loss: 0.4653 - accuracy: 0.8363 - val_loss: 0.4359 - val_accuracy: 0.8430
Epoch 6/50
333/333 [==============================] - 2s 5ms/step - loss: 0.4496 - accuracy: 0.8428 - val_loss: 0.4193 - val_accuracy: 0.8531
Epoch 7/50
333/333 [==============================] - 2s 5ms/step - loss: 0.4393 - accuracy: 0.8465 - val_loss: 0.4125 - val_accuracy: 0.8548
Epoch 

In [103]:
## Let's try with LightGBM
bgm=lgb.LGBMClassifier()
bgm.fit(X_train,Y_train)
Y_train_bgm=bgm.predict(X_train)
Y_test_bgm=bgm.predict(X_test)
acc_bgm_train=accuracy_score(Y_train,Y_train_bgm)
acc_bgm_test=accuracy_score(Y_test,Y_test_bgm)
print("Accuracy of LightGBM on train set:",acc_bgm_train)
print("Accuracy of LightGBM on test set:",acc_bgm_test)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002672 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1780
[LightGBM] [Info] Number of data points in the train set: 16606, number of used features: 21
[LightGBM] [Info] Start training from score -2.112625
[LightGBM] [Info] Start training from score -1.893473
[LightGBM] [Info] Start training from score -1.966474
[LightGBM] [Info] Start training from score -1.857720
[LightGBM] [Info] Start training from score -1.649430
[LightGBM] [Info] Start training from score -2.137330
[LightGBM] [Info] Start training from score -2.100252
Accuracy of LightGBM on train set: 0.9654341804167168
Accuracy of LightGBM on test set: 0.8892100192678227


In [107]:
#let's dump this models
joblib.dump(xgb_clf,cfg.models+'xgb_clf.joblib')
joblib.dump(ann,cfg.models+'ann.joblib')
joblib.dump(bgm,cfg.models+'bgm.joblib')
joblib.dump(scale,cfg.models+'scaler.joblib')

['../models/scaler.joblib']